<a href="https://colab.research.google.com/github/sujitpatel22/Iot-attack-Detection/blob/main/IoT_attack_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading Dependencies**

In [5]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.layers import Dropout, Lambda, Reshape, Concatenate, Flatten
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow import expand_dims
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import csv
import ipaddress
import sys

# **Loading Data**

In [6]:
# Loading the Dataset using Pandas
def load_data(filename):
  dataFrame = pd.read_csv(filename)
  return dataFrame

# **Pre-processing of Dataset**

In [7]:
def preProcess(data, num_classes):
  data["class"] = to_categorical(data["class"].astype(int), num_classes)
  data = convert_ip_to_integer(data, 'Sender_IP', 'Target_IP')

  # remove any row having ["nan","infinity","null"," ","NaN"] in it.
  for index, row in data.iterrows():
    for col_name in data.columns:
      if row[col_name] in ["nan","infinity","null"," ","NaN"]:
        data = data.drop(index)
        break;

  # Convert the non-numeric columns in to int64/numeric values
  data = convert_id_to_int(data)
  data['ID'] = data['ID'].astype(int)
  # Apply MinMaxScaler to numeric columns
  numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
  scaler = MinMaxScaler()
  data[numeric_columns] = scaler.fit_transform(data[numeric_columns])
  data.to_csv('Datasets_refined.csv', header = False, index=False)

# Convert the IP_addresses columns into numeric
def convert_ip_to_integer(data, Sender_IP, Target_IP):
  data[Sender_IP] = data[Sender_IP].apply(lambda x: int(ipaddress.IPv4Address(x)))
  data[Target_IP] = data[Target_IP].apply(lambda x: int(ipaddress.IPv4Address(x)))
  return data

# Convert the ID column into numeric column of integer
def convert_id_to_int(data):
  for index, row in data.iterrows():
    splited_id = str(row["ID"]).split("-")
    converted_id = []
    for token in splited_id:
      converted_token = int(ipaddress.IPv4Address(token)) if not token.isdigit() else int(token)
      converted_id.append(converted_token)
    merged_id, pow = 0, 0
    for c_id in converted_id:
      merged_id += c_id*(256**pow)
      pow += 1
    data.at[index, "ID"] = int(merged_id)
  return data



# Spliting the  dataset into X & Y values

In [8]:
# Spliting the Preprocessed Dataset into X input data and Y truth labels
# returning X, Y as numpy arrays.
def load_processed_data(processed_csv_name):
  data = pd.read_csv(processed_csv_name, header = None)
  X = []
  Y = []
  X = data.iloc[:, :-1]  # Select all columns except the last one
  Y = data.iloc[:, -1] # Select the last column as target label column
  return np.array(X),np.array(Y)

# **Building the Model**

In [9]:
# Creating the DNN model
def create_dnn_model(num_features):
  model = Sequential()
  model.add(Dense(400, activation = "relu", kernel_initializer="he_normal", name = "dense1"))
  model.add(Dense(100, activation = "relu", kernel_initializer="he_normal", name = "dense2"))
  model.add(Dense(50, activation = "relu", kernel_initializer="he_normal", name = "dense3"))
  model.add(Flatten())
  return model

# Creating the LSTM model
def create_lstm_model(num_features):
  model = Sequential()
  model.add(LSTM(400, return_sequences=True, name="lstm1"))
  model.add(LSTM(100, return_sequences=True, name="lstm2"))
  model.add(LSTM(50, return_sequences=False, name="lstm3"))
  return model

# Main building function to merge the DNN & LSTM models to build the main model
def build_main_model(num_features, num_classes):
  # Input layer to feed data into the DNN & the LSTM model
  input_layer = Input(shape = (1, num_features), dtype = 'float32', name="input_layer")

  # Create DNN & LSTM models
  dnn_model = create_dnn_model(num_features)
  lstm_model = create_lstm_model(num_features)

  # define output layers for both the models
  dnn_output = dnn_model(input_layer)
  # print("dnn_ouput:", dnn_output.shape)
  lstm_output = lstm_model(input_layer)
  # print("lstm_ouput:", lstm_output.shape)

  # Merge both the model's outputs to one single output with same tensor shape as the individual tensor shape
  merged_layer = Concatenate()([dnn_output, lstm_output])
  dense = Dense(40, activation = "relu", kernel_initializer="he_normal", name = "dense4")(merged_layer)
  dense = Dense(15, activation = "relu", kernel_initializer="he_normal", name = "dense5")(dense)
  # Using Singmoid activation fucntion in the output layer for Binary_classification as in our specific case
  output_layer = (Dense(1, activation = "sigmoid", name="output_layer"))(dense)

  model = Model(inputs = input_layer, outputs = output_layer)
  return model

# **Compile Model**

In [10]:
# Compile the model using 'adam' optimizer and 'binary_crossentropy' loss
def compile_model(model):
  model.compile(
      optimizer = "adam",
      loss = 'binary_crossentropy',
      metrics=['accuracy']
  )
  return model

# **Evaluating Model**

In [11]:
def evaluate(model, Y_train, Y_test):
  accuracy = model.evaluate(Y_train, Y_test)
  accuracy = "{:.3f}".format(accuracy[1] * 100)
  return accuracy

# **The main driver function**

In [14]:
# NUM_FEATURES = no of features in the input_data vector
# NUM_CLASSES = no of classes for classification
NUM_FEATURES, NUM_CLASSES = 18, 2

def main():
  data = load_data("Datasets.csv")
  preProcess(data, NUM_CLASSES)
  X, Y = load_processed_data("Datasets_refined.csv")

  # Convert X data and Y labels of dtype =  np.float32
  X = X.astype(np.float32)
  Y = Y.astype(np.float32)
  X = X.reshape((X.shape[0], 1, X.shape[1]))
  # print(X.shape)
  # Spliting the X, Y data into 90% training data & 10% Testing data
  X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.10)

  model = build_main_model(NUM_FEATURES, NUM_CLASSES)
  model = compile_model(model)
  # Model is trained for 20 Epochs for even better prediction.
  model.fit(X_train, Y_train, epochs = 20, verbose = 2)

  # saving the model
  model.save("model.h5")
  # Evaluating the model on the testing dataset

  accuracy = evaluate(model, X_test, Y_test)
  print("Overall Model accuracy: ", accuracy, "%")

if __name__=="__main__":
  main()

input_layer_shape: (None, 1, 18)
Epoch 1/20
154/154 - 6s - loss: 0.4813 - accuracy: 0.7638 - 6s/epoch - 41ms/step
Epoch 2/20
154/154 - 2s - loss: 0.3843 - accuracy: 0.7937 - 2s/epoch - 15ms/step
Epoch 3/20
154/154 - 2s - loss: 0.3684 - accuracy: 0.7973 - 2s/epoch - 12ms/step
Epoch 4/20
154/154 - 2s - loss: 0.3332 - accuracy: 0.8032 - 2s/epoch - 12ms/step
Epoch 5/20
154/154 - 2s - loss: 0.3199 - accuracy: 0.8193 - 2s/epoch - 10ms/step
Epoch 6/20
154/154 - 2s - loss: 0.3096 - accuracy: 0.8286 - 2s/epoch - 11ms/step
Epoch 7/20
154/154 - 2s - loss: 0.2987 - accuracy: 0.8280 - 2s/epoch - 11ms/step
Epoch 8/20
154/154 - 2s - loss: 0.2824 - accuracy: 0.8412 - 2s/epoch - 11ms/step
Epoch 9/20
154/154 - 2s - loss: 0.2766 - accuracy: 0.8406 - 2s/epoch - 14ms/step
Epoch 10/20
154/154 - 2s - loss: 0.2734 - accuracy: 0.8475 - 2s/epoch - 12ms/step
Epoch 11/20
154/154 - 2s - loss: 0.2640 - accuracy: 0.8505 - 2s/epoch - 11ms/step
Epoch 12/20
154/154 - 2s - loss: 0.2621 - accuracy: 0.8515 - 2s/epoch - 11

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


18/18 [==============================] - 1s 4ms/step - loss: 0.2164 - accuracy: 0.8814
Overall Model accuracy:  88.139 %
